## This is the data cleaning operations we did for the kc housing bake off

##### Import all of the packages

In [12]:
import matplotlib.pyplot as plt
from math import exp
from scipy.stats import norm
from scipy import stats
import seaborn as sns
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 300)
df_kc = pd.read_csv('kc_house_data_train.csv')

##### Imputing data

In [19]:
def kc_cleaning_pipeline(dataframefile):
    df_kc = pd.read_csv(dataframefile)
    
    rnmdate = {'date':'selldate'}
    df_kc.rename(columns = rnmdate, inplace = True)
    
    df_kc.selldate = df_kc.selldate.apply(lambda x: x[:8])
    df_kc.selldate = df_kc.selldate.apply(lambda x: pd.to_datetime(x, yearfirst=True, format='%Y%m%d'))
    
    df_kc['yr_old'] = np.where(df_kc['yr_renovated'] != 0, df_kc['selldate'].apply(lambda x: x.year) - df_kc['yr_renovated'], 
                       df_kc['selldate'].apply(lambda x: x.year) - df_kc['yr_built'])

    df_kc['yr_old'] = np.where(df_kc['yr_old'] < 0, 0, df_kc['yr_old'])
    
    # turning the 0s into nans 
    df_kc.yr_renovated = np.where(df_kc.yr_renovated == 0, np.nan, df_kc.yr_renovated)

    # make a column that has the sqft living / sqft lot. Effectively, how big is the house in comparison to the land

    df_kc['liv_lot_ratio'] = df_kc.sqft_living / df_kc.sqft_lot 



    """ 
    The below np.select function is to adjust all of the houses estimation of number of bathrooms 

    """


    conditions = [  (df_kc.sqft_living <= 3000) & ((df_kc.bathrooms >= 0) & (df_kc.bathrooms <= 1.5)),
                    ((df_kc.sqft_living  <= 3800) & (df_kc.sqft_living  >= 3000))  & ((df_kc.bathrooms >= 0) & (df_kc.bathrooms <= 3)), 
                    ((df_kc.sqft_living  <= 4800) & (df_kc.sqft_living  > 3800))  & ((df_kc.bathrooms >= 0) & (df_kc.bathrooms <= 2)),
                   (df_kc.liv_lot_ratio <= 0.001)]

    choices = [ df_kc.bathrooms,
                df_kc.bathrooms.median(),
                df_kc.bedrooms,
                df_kc.bathrooms]

    df_kc.bathrooms = np.select(conditions, choices, default = df_kc.bathrooms)

    """
    The below np.select() function is to adjust all of the houses with an overestimation of bedrooms
    """


    df_kc.at[8597, 'bedrooms'] = 3

    conditions = [ (df_kc.sqft_living <= 3000) & ((df_kc.bedrooms >= 1) & (df_kc.bedrooms <= 7)), # do the smaller end
                    (df_kc.sqft_living  > 4000) & ((df_kc.bedrooms >= 1) & (df_kc.bedrooms <= 3)) # do the upper end
                   ]

    choices = [ df_kc.bedrooms.mean(),
               5
                ]

    df_kc.bedrooms = np.select(conditions, choices, default = df_kc.bedrooms)


    """
    The below np.select() function is to adjust all of the houses with an underestimation of bedrooms
    """
    conditions = [  (df_kc.sqft_living > 3000) & ((df_kc.bedrooms >= 0) & (df_kc.bedrooms <= 3)), # do the upper end
                    ((df_kc.sqft_living  <= 3800) & (df_kc.sqft_living  > 2800))  & ((df_kc.bedrooms >= 0) & (df_kc.bedrooms <= 3)) # do the lower end
                   ]

    choices = [ 6.5,
                4.5
                ]

    df_kc.bedrooms = np.select(conditions, choices, default = df_kc.bedrooms)
    
    ### Feature Engineering section:
    
    # this gives us a living to lot size ratio
    df_kc['price_sqft_liv'] = df_kc['price']/df_kc['sqft_living'] 

    # bedrooms to bathrooms ratio
    df_kc['bed_bath'] = df_kc['bedrooms']/df_kc['bathrooms']
    
    return df_kc
    
    #### Construct metro proximity feature
    #transit_loc = {'Northgate TC': (47.707696,-122.326842),
    #           'UW': (47.651572,-122.304242) ,
    #           'Westlake': (47.612982,-122.336532) ,
    #           'Capitol Hill': (47.621353,-122.320111) ,
    #           'Bellevue': (47.617540,-122.-195230),
    #           'Pine Street, 9th ave': (47.614489,-122.332074) ,
    #           'Pioneer Square': (47.603276,-122.331904) ,
    #           'Kings Street': (47.598833,-122.329926) ,
    #           "Int'l District/ctown": (47.598308,-122.327837) ,
    #           '6th ave S & S Atlantic': (47.591008,-122.325878) ,
    #           'Rainier Ave & Mt Baker TC': (47.578748,-122.297142) ,
    #           'Rainier Beach': (47.524828,-122.280494),
    #           'Columbia City': (47.560558,-122.293057) ,
    #           'Kings County South Base': (47.499379,-122.284285) ,
    #           'Tukwila': (47.463233,-122.238660) ,
    #           'Renton TC': (47.488332,-122.210975),
    #           'Burien TC': (47.474578,-122.334315)}
    #
    #import geopy.distance
    #metro_prox = []
    #for houseloc in loc_coord:
    #    sortlist=[]
    #    for transitloc in transit_loc.values():
    #        sortlist.append(geopy.distance.great_circle(houseloc,transitloc).miles)
    #        #print(len(sortlist))
    #    metro_prox.append(min(sortlist))
    #    
    #df['metro_prox'] = metro_prox

In [3]:

# turning the 0s into nans 
#df_kc.yr_renovated = np.where(df_kc.yr_renovated == 0, np.nan, df_kc.yr_renovated)

## make a column that has the sqft living / sqft lot. Effectively, how big is the house in comparison to the land
#
#df_kc['liv_lot_ratio'] = df_kc.sqft_living / df_kc.sqft_lot 
#
#
#
#""" 
#The below np.select function is to adjust all of the houses estimation of number of bathrooms 
#
#"""
#
#
#conditions = [ ((df_kc.price <= 650000)) & (df_kc.sqft_living <= 3000) & ((df_kc.bathrooms >= 0) & (df_kc.bathrooms <= 1.5)),
#               ((df_kc.price  > 650000) & (df_kc.price <= 850000)) & ((df_kc.sqft_living  <= 3800) & (df_kc.sqft_living  >= 3000))  & ((df_kc.bathrooms >= 0) & (df_kc.bathrooms <= 3)), 
#               ((df_kc.price  > 850000) & (df_kc.price < 1250000)) & ((df_kc.sqft_living  <= 4800) & (df_kc.sqft_living  > 3800))  & ((df_kc.bathrooms >= 0) & (df_kc.bathrooms <= 2)),
#               (df_kc.liv_lot_ratio <= 0.001)]
#
#choices = [ df_kc.bathrooms,
#            df_kc.bathrooms.median(),
#            df_kc.bedrooms,
#            df_kc.bathrooms]
#
#df_kc.bathrooms = np.select(conditions, choices, default = df_kc.bathrooms)
#
#"""
#The below np.select() function is to adjust all of the houses with an overestimation of bedrooms
#"""
#
#
#df_kc.at[8597, 'bedrooms'] = 3
#
#conditions = [ (df_kc.price <= 650000) & (df_kc.sqft_living <= 3000) & ((df_kc.bedrooms >= 1) & (df_kc.bedrooms <= 7)), # do the smaller end
#               ((df_kc.price  > 700000) & (df_kc.price < 1000000) & (df_kc.sqft_living  > 4000)) & ((df_kc.bedrooms >= 1) & (df_kc.bedrooms <= 3)) # do the upper end
#               ]
#
#choices = [ df_kc.bedrooms.mean(),
#            5
#            ]
#
#df_kc.bedrooms = np.select(conditions, choices, default = df_kc.bedrooms)
#
#
#"""
#The below np.select() function is to adjust all of the houses with an underestimation of bedrooms
#"""
#conditions = [ ((df_kc.price > 850000)) & (df_kc.sqft_living > 3000) & ((df_kc.bedrooms >= 0) & (df_kc.bedrooms <= 3)), # do the upper end
#               ((df_kc.price  > 700000) & (df_kc.price < 850000)) & ((df_kc.sqft_living  <= 3800) & (df_kc.sqft_living  > 2800))  & ((df_kc.bedrooms >= 0) & (df_kc.bedrooms <= 3)) # do the lower end
#               ]
#
#choices = [ 6.5,
#            4.5
#            ]
#
#df_kc.bedrooms = np.select(conditions, choices, default = df_kc.bedrooms)
#
## this gives us a living to lot size ratio
#df_kc['price_sqft_liv'] = df_kc['price']/df_kc['sqft_living'] 
#
# # bedrooms to bathrooms ratio
#df_kc['bed_bath'] = df_kc['bedrooms']/df_kc['bathrooms']

### feature engineering

# this gives us a living to lot size ratio
#df_kc['price_sqft_liv'] = df_kc['price']/df_kc['sqft_living'] 

 # bedrooms to bathrooms ratio
#df_kc['bed_bath'] = df_kc['bedrooms']/df_kc['bathrooms']


In [11]:
df_kc.describe()

,Unnamed: 0,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,liv_lot_ratio,price_sqft_liv,bed_bath
count,17290.000000,1.729000e+04,1.729000e+04,17290.000000,17290.000000,17290.000000,1.729000e+04,17290.000000,17290.000000,17290.000000,17290.000000,17290.000000,17290.000000,17290.000000,17290.000000,726.000000,17290.000000,17290.000000,17290.000000,17290.000000,17290.000000,17290.000000,17290.000000,1.728600e+04
mean,8644.500000,4.565502e+09,5.407395e+05,3.557109,2.105451,2081.464604,1.524340e+04,1.490312,0.007981,0.238519,3.408502,7.654425,1789.306015,292.158589,1970.792019,1995.882920,98078.193175,47.560058,-122.214258,1987.986698,12873.475824,0.320342,263.854204,inf
std,4991.337413,2.874656e+09,3.733190e+05,0.618221,0.767198,920.018539,4.230462e+04,0.538909,0.088985,0.775229,0.651296,1.174718,829.265107,443.151874,29.343516,15.646837,53.607949,0.138412,0.140857,684.802635,27227.437583,0.262130,109.776947,NaN
min,0.000000,1.000102e+06,7.500000e+04,0.000000,0.000000,290.000000,5.720000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,1934.000000,98001.000000,47.155900,-122.519000,399.000000,659.000000,0.000610,87.588235,0.000000e+00
25%,4322.250000,2.114701e+09,3.210000e+05,3.370735,1.500000,1430.000000,5.081250e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1200.000000,0.000000,1951.000000,1987.250000,98033.000000,47.471200,-122.329000,1490.000000,5111.250000,0.156572,182.432432,1.348294e+00
50%,8644.500000,3.903650e+09,4.500000e+05,3.370735,2.250000,1920.000000,7.642000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1974.000000,2000.000000,98065.000000,47.571600,-122.230000,1840.000000,7622.500000,0.246534,244.316761,1.685367e+00
75%,12966.750000,7.301150e+09,6.450000e+05,3.370735,2.500000,2550.000000,1.072575e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2214.500000,560.000000,1996.000000,2007.000000,98118.000000,47.677900,-122.126000,2360.000000,10101.750000,0.402199,317.454778,2.247156e+00
max,17289.000000,9.900000e+09,7.700000e+06,11.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,858132.000000,2.187500,810.138889,inf


In [36]:
kc_df.isna().sum()

Unnamed: 0            0
id                    0
selldate              0
price                 0
bedrooms              0
bathrooms             0
sqft_living           0
sqft_lot              0
floors                0
waterfront            0
view                  0
condition             0
grade                 0
sqft_above            0
sqft_basement         0
yr_built              0
yr_renovated      16564
zipcode               0
lat                   0
long                  0
sqft_living15         0
sqft_lot15            0
yr_old                0
liv_lot_ratio         0
price_sqft_liv        0
bed_bath              4
dtype: int64

In [20]:
kc_df = kc_cleaning_pipeline('kc_house_data_train.csv')

In [5]:
kc_df.columns

Index(['Unnamed: 0', 'id', 'selldate', 'price', 'bedrooms', 'bathrooms',
       'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition',
       'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated',
       'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15', 'yr_old',
       'liv_lot_ratio', 'price_sqft_liv', 'bed_bath'],
      dtype='object')

In [21]:
features = ['bedrooms', 'bathrooms',
       'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition',
       'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_old',
       'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15',
       'liv_lot_ratio', 'price_sqft_liv']
kc_df_features = kc_df[features]
target = kc_df.price

In [22]:
X_train, X_test, y_train, y_test = train_test_split(kc_df_features, target, random_state=34, test_size=0.2)
#check the shape of the results
print("Training set - Features: ", X_train.shape, "Target: ", y_train.shape)
print("Training set - Features: ", X_test.shape, "Target: ",y_test.shape)

Training set - Features:  (13832, 20) Target:  (13832,)
Training set - Features:  (3458, 20) Target:  (3458,)


In [23]:
# fit a model only to training set
#instantiate a linear regression object
lm = LinearRegression()
#fit the linear regression to the data
lm = lm.fit(X_train, y_train)
print(lm.intercept_)
print(lm.coef_)

14203476.321746768
[-1.68444209e+04  1.41878832e+04  2.02250477e+02 -1.34266388e-01
 -1.53524751e+04  2.63080991e+05  1.18903727e+04  5.41375895e+03
  1.74401658e+04  1.04533497e+02  9.77169805e+01 -3.86363302e+02
 -2.03838098e+02 -1.45907060e+02 -5.41569723e+04 -2.27562192e+04
 -3.10358909e+01 -1.60533061e-01 -1.16620710e+04  2.05132800e+03]


In [24]:
print ("R^2 Score:", lm.score(X_train, y_train))

R^2 Score: 0.8888890321173869


In [25]:
y_train_pred = lm.predict(X_train) 
y_train_pred

array([381311.86570982, 305952.98869571, 234283.16064692, ...,
       408225.6663974 , 854003.33372026, 194539.60117012])

In [26]:
from sklearn import metrics

train_mae = metrics.mean_absolute_error(y_train, y_train_pred)
train_mse = metrics.mean_squared_error(y_train, y_train_pred)
train_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))


print('Mean Absolute Error:', train_mae )
print('Mean Squared Error:',  train_mse)
print('Root Mean Squared Error:' , train_rmse)

Mean Absolute Error: 69024.44688462494
Mean Squared Error: 15385323548.20897
Root Mean Squared Error: 124037.58925506803
